# GenAI Workshop
## Lesson 3: Basic RAG

This lesson is intended to show you the basics of a Retrieval Augmented Generation (RAG) system.

During this lesson you will learn how to ...

- implement the different building blocks of RAG
- create a ingestion pipeline from the building blocks
- create a retrieval pipeline from the building blocks
- use a RAG system to generate responses to user inputs

### Set up the environment

In [109]:
import os
import google.generativeai as genai

if os.getenv("COLAB_RELEASE_TAG"):
   COLAB = True
   print("Running on COLAB environment.")
else:
   COLAB = False
   print("WARNING: Running on LOCAL environment.")


In [ ]:
# Clone the data repository into colab
!git clone https://github.com/openknowledge/workshop-genai-data.git
PROCESSED_DATA_PATH = "/content/workshop-genai-data/processed/gutenberg/"

In [110]:
# import colab specific lib to read user data (aka colab managed secrets)
from google.colab import userdata

In [111]:
# Initialize Google GenAI Client API with GOOGLE_API_KEY to be able to call the model.
# Note: GEMINI_API_KEY must be set as COLAB userdata before!
GOOGLE_API_KEY=userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
# Install additional libraries
%%capture
!pip install -qU langchain-text-splitters
!pip install chromadb

In [ ]:
# Import additional libraries
from langchain_text_splitters import RecursiveCharacterTextSplitter
from chromadb import EphemeralClient
import uuid

In [ ]:
# Set default values for model, model parameters and prompt
DEFAULT_MODEL = "gemini-1.5-flash"
DEFAULT_CONFIG_TEMPERATURE = 0.9
DEFAULT_CONFIG_TOP_K = 1
DEFAULT_CONFIG_MAX_OUTPUT_TOKENS = 200
DEFAULT_SYSTEM_PROMPT = "Your are a friendly assistant"
DEFAULT_USER_PROMPT = " "

In [ ]:
# This will be the chromadb collection we use as a knowledge base. We do not need the in-memory EphemeralClient.
chromadb_collection = EphemeralClient().get_or_create_collection(name="default")

# Have a look into the knowledgebase
def peek_knowledgebase():
  """Shows the first ten items of the knowledgebase"""
  print(chromadb_collection.peek())

In [ ]:
def call_genai_model_for_completion(
        model_name: str = DEFAULT_MODEL,
        config_temperature:float = DEFAULT_CONFIG_TEMPERATURE,
        config_top_k: int = DEFAULT_CONFIG_TOP_K,
        config_max_output_tokens: int = DEFAULT_CONFIG_MAX_OUTPUT_TOKENS,
        system_prompt : str = DEFAULT_SYSTEM_PROMPT,
        user_prompt : str = DEFAULT_USER_PROMPT,
        verbose: bool = False
        ):
    """ Calls a gemini model with a given set of parameters and returns the completions

    Parameters
    ----------
    model_name : str, optional [default: DEFAULT_MODEL]
        The name of the model to use for the completion
    temperature : float, optional [default: DEFAULT_CONFIG_TEMPERATURE]
        The temperature of the model
    top_k : int, optional [default: DEFAULT_CONFIG_TOP_K]
        The number of most recent matches to return
    max_output_tokens : int, optional [default: DEFAULT_CONFIG_MAX_OUTPUT_TOKENS]
        The maximum number of output tokens to return
    system_prompt : str, optional [default: DEFAULT_SYSTEM_PROMPT]
        The system prompt to use for the completion
    user_prompt : str, optional [default: DEFAULT_USER_PROMPT]
        The user prompt to use for the completion
    file_list : [str], optional [default: empty list]
    verbose : bool, optional [default: False]
        Whether to print details of the completion process or not. Defaults to False
    Returns
    -------
    completions :
        a GenerateContentResponse instance representing the genAI model answer(s)
    """
    if verbose:
        # print out summary of input values / parameters
        print(f'Generating answer for following config:')
        print(f'  - SYSTEM PROMPT used:\n {system_prompt}')
        print(f'  - USER PROMPT used:\n {user_prompt}')
        print(f'  - MODEL used:\n {model_name} (temperature = {config_temperature}, top_k = {config_top_k}, max_output_tokens = {config_max_output_tokens})')

    # create generation config
    model_config = genai.GenerationConfig(
        max_output_tokens=config_max_output_tokens,
        temperature=config_temperature,
        top_k=config_top_k
    )

    # create genai model with generation config
    genai_model = genai.GenerativeModel(
        model_name= model_name,
        generation_config= model_config
    )

    response = genai_model.generate_content([system_prompt, user_prompt])
    return response

In [ ]:
def print_completion_result(completion_result, full:bool = False):
    """ Prints the completion result of a genAI model

    Parameters
    ----------
    completion_result :
        a GenerateContentResponse instance representing the genAI model answer(s)
    full : bool, optional [default: False]
        Whether to print the full completion result or not. Defaults to False
    """
    # print out answer of genai model (aka text of response)
    print(f'\nANSWER of genAI model: \n')
    if full:
        print(completion_result)
    else:
        print(completion_result.text)

### Configure the genAI models

In [118]:
GENERATION_MODEL = "gemini-1.5-flash"
EMBEDDING_MODEL = "models/text-embedding-004"

### Configure retriever

In [133]:
DEFAULT_K = 3
DEFAULT_CHUNK_SIZE = 2000
DEFAULT_CHUNK_OVERLAP = 100

### Define RAG Building Blocks

In [ ]:
# Get content of books. The content will already be cleansed.
def load_file_content(file_name: str) -> str:
  """ Loads content of a file in the local file systemby a given file name and return its content

    Parameters
    ----------
    file_name : str
        The name of the file to be loaded
    Returns
    -------
    file_content : str
        the content of the file as a string
  """
  with open(f"{PROCESSED_DATA_PATH}{file_name}", "r") as f:
    return f.read()

In [ ]:
# Building Block "Chunking": Split the content into smaller chunks
def do_chunk(text: str) -> list[str]:
  """ Chunks a given text by a given chunk size and chunk overlap and returns a list of chunks

  Parameters
  ----------
  text : str
      The text to be chunked
  chunk_size : int, optional [default: DEFAULT_CHUNK_SIZE]
        The desired chunk size
  chunk_overlap : int, optional [default: DEFAULT_CHUNK_OVERLAP]
        The desired chunk overlap
  Returns
  -------
  chunks: [str]
      The created chunks
  """
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=DEFAULT_CHUNK_SIZE,
      chunk_overlap=DEFAULT_CHUNK_OVERLAP,
      length_function=len,
  )
  return text_splitter.split_text(text=text)

In [ ]:
# Building Block "Embedding": Create multi dimensional embeddings for a given chunk.
def do_embed(chunk: str) -> list[float]:
  """ Embeds a given chunk and returns the embedding

  Parameters
  ----------
  chunk : str
      The chunk to be embedded
  Returns
  -------
  embedding: [float]
      The created embedding
  """
  return genai.embed_content(model=EMBEDDING_MODEL, content=chunk).get("embedding")

def do_batch_embed(chunks: list[str]) -> list[list[float]]:
  """ Embeds a list of chunks and returns the embeddings

  Parameters
  ----------
  chunks : [str]
      The chunks to be embedded
  Returns
  -------
  embeddings: [list[float]]
      The created embeddings
  """
  return genai.embed_content(model=EMBEDDING_MODEL, content=chunks).get("embedding")

In [ ]:
# Building Block "Knowledgebase": Store embeddings and the corresponding content in a vectorstore
def persist_embeddings(chunks: list[str], embeddings: list[float])-> None:
  """ Persists the embeddings and the chunks in the knowledgebase

  Parameters
  ----------
  chunks : [str]
      The chunks to be persisted
  embeddings: [list[float]]
      The corresponding embeddings to be persisted
  """
  ids = [str(uuid.uuid4()) for _ in chunks]
  chromadb_collection.add(ids=ids, documents=chunks, embeddings=embeddings)

In [ ]:
# Building Block "Augmentation": Create an updated prompt by merging the original user input with the provided context
def augment(user_input: str, context: list[str]) -> str:
  """ Augments a given user input by merging it with the provided context and returns the augmented prompt

  Parameters
  ----------
  user_input : str
      The user input to be augmented
  context : [str]
      The context to be merged with the user input
  Returns
  -------
  augmented_prompt: str
      The created augmented prompt
  """
  prepared_context = "\n".join(context)
  augmented_prompt = f"""
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n{prepared_context}?\n
    Question: \n{user_input}\n

    Answer:
  """
  return augmented_prompt

In [ ]:
# Building Block "Top-k Fetching": Get the k semantically closest chunks to the user input from the knowledgebase
def do_top_k_fetching(user_input_embedding: list[float], top_k: int) -> list[str]:
  """ Fetches the k semantically closest chunks to the user input from the knowledgebase

  Parameters
  ----------
  user_input_embedding : [float]
      The embedding of the user input
  top_k : int
      The number of semantically closest chunks to be fetched

  Returns
  -------
  context: [str]
      The fetched chunks
  """
  # Since we will do the fetching always only for one user_input,
  # instead of querying for multiple embeddings simultanously as allowed by the choma API,
  # we add the embeddings below to a list and return only the first document (chunk)
  return chromadb_collection.query(
      query_embeddings=[user_input_embedding],
      n_results=top_k,
  )["documents"][0]

In [ ]:
# Building Block "Generation": Use the generation model to create a response
def generate_response(prompt: str) -> None:
  """ Generates a response for a given prompt

  Parameters
  ----------
  prompt : str
      The prompt to be used for the generation
  """
  completion_result = call_genai_model_for_completion(
      model_name=GENERATION_MODEL,
      user_prompt=prompt,
  )
  print_completion_result(completion_result)

### Create the ingestion pipeline

In [ ]:
def do_ingestion(file_names: list[str]) -> None:
  """ Ingests a list of files by a given file name

  Parameters
  ----------
  file_names : [str]
      The names of the files to be ingested
  """
  # Ingest file by file
  for file_name in file_names:
    # Load prepared book content
    file_content = load_file_content(file_name)

    # Chunk the content into smaller chunks
    chunks = do_chunk(file_content)

    # Embed the chunks
    embeddings = do_batch_embed(chunks)

    # Persist the embeddings and the chunks in the knowledgebase
    persist_embeddings(chunks, embeddings)


### Perform ingestion

In [127]:
# Define file names to be ingested
file_names = ['study_in_scarlett.txt']

# Perform ingestion. Depending on the chunk_size this might take some minutes.
do_ingestion(file_names)

In [132]:
# Use helper function to peek into knowledgebase
peek_knowledgebase()

### Exercise 01: Create RAG pipeline
In this exercise you will create a rag pipeline for retrieving relevant chunks and generating a grounded response

In [ ]:
def do_rag(user_input: str, verbose: bool = False) -> None:
  """ Runs the RAG pipeline with a given user input and prints the response

  Parameters
  ----------
  user_input : str
      The user input to be used for the RAG pipeline
  verbose : bool, optional [default: False]
      Whether to print details of the RAG process or not. Defaults to False
  """
    
  # TODO: Embed the user input
  user_input_embedding = None

  # TODO: "R" like "Retrieval": Get the k semantically closest chunks to the user input from the knowledgebase
  context = None
  if verbose:
    print(f'Retrieved context:\n {context}')

  # TODO: "A" like "Augmented": Create the augmented prompt
  augmented_prompt = None
  if verbose:
    print(f'Augmented prompt:\n {augmented_prompt}')

  # TODO: "G" like "Generation": Generate a response


In [ ]:
# Define user input. This should be a question regarding the ingested book
user_input = "Lucy noticed a number on the ceiling when taking breakfast. which number was written into the ceiling?" # The answer should contain the number "28"

# TODO: Perform rag
